<a href="https://colab.research.google.com/github/SakaGloria/FMMLLABS_AND_PROJECTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


The size of the training and validation sets can influence how well the model's performance on the validation set predicts its performance on the unseen test set. Here are some considerations:

### **1. Sufficient Data for Learning:**
   - **Training Set Size:**
     - A larger training set generally allows the model to learn more robust and generalized patterns from the data.
     - If the training set is too small, the model may struggle to capture the underlying patterns, leading to poor generalization.

   - **Validation Set Size:**
     - A sufficiently large validation set is crucial for accurately estimating the model's performance. It should be representative of the data distribution and diverse enough to provide a reliable evaluation.

### **2. Overfitting and Underfitting:**
   - **Overfitting:**
     - If the model performs exceptionally well on a small validation set, it might be overfitting to the validation data, capturing noise instead of true patterns. This can result in poor generalization to the test set.

   - **Underfitting:**
     - Conversely, if the model performs poorly on the validation set, it may not have learned the underlying patterns well, indicating potential underfitting. In such cases, performance on the test set may also be suboptimal.

### **3. Generalization Estimate:**
   - **Validation Set as a Proxy for Test Set:**
     - The validation set is typically used as a proxy for the test set during model development. A good performance on the validation set suggests that the model is likely to generalize well to unseen data.

   - **Caution with Small Validation Sets:**
     - If the validation set is small, the estimate of model performance may be less reliable. The risk of overfitting to the specific characteristics of the validation set increases.

### **4. Cross-Validation:**
   - **Using Multiple Validation Sets:**
     - Cross-validation, which involves splitting the data into multiple train-validation sets, can provide a more robust estimate of the model's generalization performance.
     - It helps mitigate the impact of variability in a single validation set and provides a more reliable indication of how well the model is likely to perform on unseen data.

### **5. Model Complexity and Training Duration:**
   - **Complex Models:**
     - More complex models may require larger datasets for effective training. If the model is too complex for the available data, it may struggle to generalize.

   - **Training Duration:**
     - The time required to train the model can also be a factor. If training takes a long time, a smaller dataset might limit the number of iterations, potentially impacting convergence.

### **Conclusion:**
In summary, having sufficiently large and representative training and validation sets is essential for building models that generalize well to unseen data. The validation set's role is to provide a reliable estimate of how well the model is likely to perform on the test set. Cross-validation can enhance the robustness of this estimate, especially when dealing with limited data. Balancing the sizes of the training and validation sets while considering the model's complexity and the nature of the problem is crucial for achieving reliable generalization.

The size of the validation set can have an impact on the accuracy estimate and generalization performance of your machine learning model. Here's how increasing or decreasing the percentage of the validation set may affect your results:

### Increasing the Percentage of Validation Set:

1. **Advantages:**
   - **More Robust Evaluation:** A larger validation set provides a more robust evaluation of the model's performance. It is less susceptible to variability introduced by a small sample of validation data.

   - **Better Generalization Estimate:** With more validation samples, the estimate of how well the model generalizes to new, unseen data tends to be more reliable.

2. **Considerations:**
   - **Reduced Training Data:** Allocating a larger percentage to the validation set means less data available for training. If the training set becomes too small, the model might not learn robust patterns, and overfitting to the training set becomes a concern.

   - **Computational Cost:** Training on a smaller training set may lead to faster training times, but it might also limit the model's ability to capture the complexity of the underlying patterns.

### Decreasing the Percentage of Validation Set:

1. **Advantages:**
   - **More Training Data:** Allocating a smaller percentage to the validation set allows more data to be used for training. This can be advantageous, especially with limited data, as it helps the model to learn better representations.

   - **Computational Efficiency:** With a larger training set, training times may be shorter, and the model may still achieve good generalization if the data is representative.

2. **Considerations:**
   - **Less Reliable Evaluation:** A smaller validation set may result in a less reliable estimate of the model's performance. The estimate becomes more susceptible to the specific characteristics of the validation samples.

   - **Risk of Overfitting:** With a smaller validation set, there is an increased risk of overfitting to the validation set, and the estimate of generalization performance may be overly optimistic.

### Finding the Right Balance:

The key is to find a balance between having a sufficiently large validation set for reliable performance estimation and having enough training data for the model to learn effectively. Cross-validation can be a valuable technique to mitigate the impact of variability in the validation set by repeatedly splitting the data into training and validation sets.

In practice, the optimal size of the validation set depends on the size and nature of the overall dataset, the complexity of the model, and the specific problem at hand. It often involves a trade-off between reliable performance estimation and effective model training.

Increasing the number of iterations may help to some extent when dealing with a very small training dataset, but it comes with certain considerations and limitations.

### Advantages of Increasing Iterations with a Small Training Dataset:

1. **More Exposure to Data:**
   - With a small dataset, each iteration allows the model to see the training examples again, potentially reinforcing patterns and improving learning.

2. **Improved Convergence:**
   - A small dataset might converge more quickly, and increasing iterations can allow the model to continue refining its parameters.

### Considerations and Limitations:

1. **Overfitting Risk:**
   - Increasing iterations with a small dataset raises the risk of overfitting. The model may memorize the training examples rather than learning the underlying patterns.

2. **Generalization Concerns:**
   - The primary goal in machine learning is to build models that generalize well to new, unseen data. If the dataset is very small, the model might not learn robust representations, and overfitting becomes a significant concern.

3. **Data Augmentation and Regularization:**
   - Instead of relying solely on increasing iterations, it's often more effective to use techniques like data augmentation (if applicable) and regularization to enhance model generalization.

4. **Model Complexity:**
   - The model's complexity (e.g., the number of parameters) should be carefully chosen. A complex model may overfit small datasets more easily, and increasing iterations may exacerbate this issue.

5. **Validation Dataset Size:**
   - If the validation dataset is also small, the estimate of model performance may be less reliable. Cross-validation with small datasets becomes challenging, and the risk of overfitting to a specific split increases.

In conclusion, while increasing iterations can be a strategy to extract more information from a small dataset, it should be approached with caution. Combining it with proper regularization, model complexity control, and other techniques (such as data augmentation) is essential to prevent overfitting and improve generalization. Additionally, the overall effectiveness depends on the specific characteristics of the dataset and the nature of the problem being addressed.

The number of iterations, or epochs, in the context of training a machine learning model, can impact the performance estimate. The effect of the number of iterations on the estimate depends on various factors, and there is no one-size-fits-all answer. Here are some considerations:

1. **Underfitting and Convergence:**
   - If your model is underfitting (i.e., it hasn't learned the underlying patterns in the data), increasing the number of iterations may improve performance up to a certain point.
   - Training for too few iterations might result in a model that hasn't converged to a good solution, leading to suboptimal performance.

2. **Overfitting:**
   - On the other hand, if you train for too many iterations, the model may start to overfit the training data. It might memorize the training set instead of learning the underlying patterns, leading to poor generalization to new, unseen data.
   - Regularization techniques (like dropout or L2 regularization) can help mitigate overfitting to some extent.

3. **Computational Resources:**
   - The number of iterations also depends on computational resources. Training for a large number of iterations may be computationally expensive, and there might be diminishing returns in terms of model improvement.

4. **Early Stopping:**
   - To find a good balance, practitioners often use techniques like early stopping. This involves monitoring the performance on a validation set during training and stopping the training process when performance starts to degrade (indicating potential overfitting).

5. **Learning Rate:**
   - The learning rate, which determines the size of the steps taken during optimization, can interact with the number of iterations. Too high of a learning rate may lead to overshooting optimal parameter values, while too low of a learning rate may result in slow convergence.

In summary, the relationship between the number of iterations and the performance estimate is complex. It involves finding a balance between underfitting and overfitting, considering computational constraints, and often involves hyperparameter tuning. Cross-validation can help in assessing the impact of different numbers of iterations on model performance across multiple folds of the data, providing insights into the generalization ability of the model.

Cross-validation, by providing a more robust estimate of model performance on the validation set, can give a more accurate estimate of how well a model is likely to perform on unseen data, such as a test set. However, it's important to note that the test accuracy is not directly estimated by cross-validation.

In a typical machine learning workflow, you use cross-validation on your training data to tune hyperparameters, select models, and assess performance. Once you've finalized your model based on these evaluations, you then evaluate it on a separate and previously unseen test set. The test set is crucial for providing an unbiased and final estimate of the model's generalization performance.

Cross-validation helps in providing a more reliable estimate of how well your model generalizes to different subsets of the training data. This can give you confidence that your model is not overfitting to a particular training-validation split. However, the final measure of accuracy or performance should come from evaluating the model on the test set.

In summary, cross-validation aids in model selection and hyperparameter tuning, leading to a more reliable and generalizable model. But the ultimate measure of accuracy on unseen data is obtained by evaluating the model on a separate test set that was not used during training or cross-validation.

Yes, averaging validation accuracy across multiple splits, often referred to as cross-validation, can provide more consistent and reliable results compared to a single split. Cross-validation is a technique used in machine learning to assess the performance of a model by splitting the dataset into multiple subsets (folds). The model is trained on several combinations of training and validation sets, and the performance metrics are averaged across these folds.

The main advantages of cross-validation include:

1. **Reduced Variance:** Averaging results over multiple folds helps reduce the variance in performance estimation. This is particularly important when the dataset is limited, and a single split may not be representative of the overall model performance.

2. **Better Generalization:** Cross-validation provides a more robust estimate of how well a model generalizes to unseen data. By training and validating on different subsets of the data, the model is exposed to a variety of patterns and variations present in the dataset.

3. **Mitigating Overfitting or Selection Bias:** Averaging results over multiple folds helps mitigate the risk of overfitting to a specific subset of the data. It also reduces the chance of accidentally selecting a biased subset that might not be representative of the overall dataset.

4. **More Reliable Performance Metrics:** The variability in model performance metrics is better captured through cross-validation, leading to more reliable and stable estimates of accuracy, precision, recall, etc.

However, it's essential to note that the choice of the number of folds can impact the trade-off between computational cost and the reliability of the results. Common choices include 5-fold or 10-fold cross-validation, but the optimal number may depend on the size of the dataset and the specific problem at hand.

In summary, averaging validation accuracy across multiple splits through cross-validation is generally a good practice for obtaining more consistent and reliable performance estimates for machine learning models.